# Adding a service demand by correlation

In the previous section we added an exogenous service demand. That is, we explicitly specified what the demand would be per year. 
However, we may not know what the electricity demand may be per year. Instead, we may conclude that our electricity demand is a function of the GDP and population of a particular region. 
To accommodate such a scenario, MUSE enables us to choose a regression function that estimates service demands from GDP and population, which may be more certain in your case.
In this section we will show how this can be done.

## Additional files

For this work, we will use the `default` example from the MUSE repository, and will not build on the previous examples. This is done to simplify the model at this point.

You can copy the files for the default model in order to edit them like so:

```bash
python -m muse --model default --copy PATH/TO/COPY/THE/MODEL/TO
```

Similarly to before, we must amend the `technodata/preset` folder for this. As we are no longer explicitly specifying demand, we can delete the `Residential2020Consumption.csv` and `Residential2050Consumption.csv` files. Instead, we must replace these files with the following:

- A macrodrivers file. This contains the drivers of the service demand that we want to model. For this example, these will include GDP based on purchasing power parity (GDP PPP) and the population that we expect from 2010 to 2110.

- A regression parameters file. This file will set the function type we would like to use to predict the service demand and the respective parameters of this regression file per region.

- A timeslice share file. This file sets how the demand is shared between timeslice.

The example files for each of those just mentioned can be found below, respectively:

- [Macrodrivers.csv](https://github.com/EnergySystemsModellingLab/MUSE_OS/blob/main/docs/tutorial-code/6-add-correlation-demand/1-correlation/technodata/preset/Macrodrivers.csv)

- [regressionparameters.csv](https://github.com/EnergySystemsModellingLab/MUSE_OS/blob/main/docs/tutorial-code/6-add-correlation-demand/1-correlation/technodata/preset/regressionparameters.csv)

- [TimesliceSharepreset.csv](https://github.com/EnergySystemsModellingLab/MUSE_OS/blob/main/docs/tutorial-code/6-add-correlation-demand/1-correlation/technodata/preset/TimesliceSharepreset.csv)

For a full introduction to these files, see the link [here](../inputs/correlation_files.rst).

Download these files and save them within the `preset` folder.

Next, we must amend our toml file to include our new way of calculating the preset service demand.

## TOML file

Editing the TOML file to include this can be done relatively quickly if we know the variable names.

In the second bottom section of the toml file, you will see the following section: 

```toml
[sectors.residential_presets]
type = 'presets'
priority = 0
consumption_path= "{path}/technodata/preset/*Consumption.csv"
```

This enables us to run the model in exogenous mode, but now we would like to run the model from the files previously mentioned. This can be done by linking new variables to the new files, as follows:

```toml
[sectors.residential_presets]
type = 'presets'
priority = 0
timeslice_shares_path = '{path}/technodata/preset/TimesliceSharepreset.csv'
macrodrivers_path = '{path}/technodata/preset/Macrodrivers.csv'
regression_path = '{path}/technodata/preset/regressionparameters.csv'
```

We effectively linked the new files to MUSE.

## Running and visualising our new results

With those changes made, we are now able to run our modified model, with the `python -m muse settings.toml` command in the command line, as before.

As before, we will now visualise the output.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
mca_capacity = pd.read_csv(
    "../tutorial-code/6-add-correlation-demand/1-correlation/Results/MCACapacity.csv"
)
mca_capacity.head()

In [ ]:
fig, axes = plt.subplots(1, 3)
all_years = mca_capacity["year"].unique()
for ax, (sector_name, sector_data) in zip(axes, mca_capacity.groupby("sector")):
    sector_capacity = sector_data.groupby(["year", "technology"]).sum().reset_index()
    sector_capacity.pivot(
        index="year", columns="technology", values="capacity"
    ).reindex(all_years).plot(kind="bar", stacked=True, ax=ax)
    ax.set_ylabel("Capacity (PJ)")
    ax.set_xlabel("Year")
    ax.set_title(f"{sector_name.capitalize()} Sector:", fontsize=10)
    ax.legend(title=None, prop={"size": 8})
    ax.tick_params(axis="both", labelsize=8)

fig.set_size_inches(8, 2.5)
fig.subplots_adjust(wspace=0.5)

As expected, we see a scenario emerge with much higher capacity limits. The demand does not increase linearly, with variations in the total demand in the residential sector. This is due to the new function not being a linear function.

## Next steps

In the next section we will see how we can enforce outputs of technologies by timeslice. For instance, we can prevent solar photovoltaics from producing electricity at night, or ensure that a nuclear power plant runs at a minimum capacity during the day.